# Accesing watsonx.ai via REST API

In this lab, we will look into making HTTP requests to access [watsonx.ai's REST API](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html) (**note**: this is technically a python library that replicates the API) and learn how to use the functionality.  This lab explores only the generate REST endpoint. One hopes that this GA product will soon have a complete documented REST API. For this notebook, the `.env` file should have a two keys: `IBMCLOUD_API_KEY` that can be created using the [IBM Cloud console](https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui#create_user_key) and `PROJECT_ID` which corresponds to the watsonx.ai project that will be used for the inference.

In [ ]:
import json
import os
import requests
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

## HTTP request headers
Headers contain parameter values that represent the metadata associated with an API requests and response. In the watsonx.ai SaaS service, an authorization token generated by the IAM service is required when making calls to the REST API. In the following example, the input credentials are used to obtain a token. This token is used to create an 'Authorization' header to validate your access using a "Bearer" token. Note that tokens requested in this way have a 1 hour lifetime.

The 'Content-type' header in the request is added to tell the server or the browser which is serving the resource to the end user about the media type of the request. In this case, type of expected data as 'application/json'.

In [ ]:

api_key = ""
api_endpoint = "https://us-south.ml.cloud.ibm.com/ml/v1-beta/"
project_id = ""
if api_key is None or project_id is None:
    print("Ensure the .env file contains a IBM Cloud API key and watsonx.ai project id.")
else:
    authenticator = IAMAuthenticator(api_key)
    access_token = authenticator.token_manager.get_token()
    headers={"Authorization": f"Bearer {access_token}",
            "Content-Type":"application/json"}
print(json.dumps(headers, indent=2))

## POST vs GET
HTTP requests come in two flavors: GET and POST.  When using GET, data parameters are included in the URL and visible to everyone. However, when using POST, data is not displayed in the URL but is instead passed in the HTTP message body. 

GET requests are intended to retrieve data from a server and do not modify the server’s state. On the other hand, POST requests are used to send data to the server for processing and may modify the server’s state.  

## POST requests with 'Generate' endpoint

The generate endpoint "{API_ENDPOINT}generation/text" provides an interface for sending prompts to any model supported by watsonx.ai. The endpoint takes a parameter of `version` that is currently only `2023-05-29`. Given a text prompt as inputs, and required parameters, the selected model will attempt to complete the provide input and return "generated_text".

Request body needs to include:
- Model id (string): the id of the model
- Inputs (string): prompt to generate completion
- Parameters for the model (key-value pairs)
- Project id (string): the id of the project to use


In [ ]:
body={
  "model_id": "google/flan-ul2",
  "input": "Write a short blog post for an advanced cloud service for large language models: This service is",
  "parameters": {
      "decoding_method": "greedy",  
      "max_new_tokens": 50,  
      "min_new_tokens": 25
  },
  "project_id": project_id
}

In [ ]:
url=f"{api_endpoint}generation/text?version=2023-05-29" 
response=requests.post(url=url, headers=headers, json=body )
print("JSON Response: ", response.json())

In [ ]:
print(f"Model results: {json.dumps(response.json()['results'], indent = 2)}") 

### Lab Complete
At the present time, using the [python library](https://ibm.github.io/watson-machine-learning-sdk/foundation_models.html) is the best window into the API for Foundation Models on watsonx.ai SaaS GA.